##### Create Spark Session

In [0]:
from pyspark.sql import SparkSession

In [0]:
spark = SparkSession.builder.appName('PROJSVC').getOrCreate()

In [0]:
df = spark.read.csv('/FileStore/tables/collision_project.csv',inferSchema=True,header=True)

#### Data pre-processing

In [0]:
# Import the required libraries
import pyspark.sql.functions as f

In [0]:
# Drop Date Column from dataset
data = spark.sql("select AVE_HUMUDITY,AVE_PRESSURE,AVE_TEMP,AVE_WIND_SPEED,DAY_OF_WEEK,COLLISION from collision")

data.show()

+------------+------------+-----------+--------------+-----------+---------+
AVE_HUMUDITY|AVE_PRESSURE| AVE_TEMP|AVE_WIND_SPEED|DAY_OF_WEEK|COLLISION|
+------------+------------+-----------+--------------+-----------+---------+
 36.875| 1027.291667|291.0070833| 0.791666667| Sunday| 98|
 70.83333333| 1011.25|298.9416667| 2.083333333| Tuesday| 153|
 71.58333333| 1015.791667|291.4237228| 1.875| Monday| 110|
 69.20833333| 1011.833333|289.1639873| 1.541666667| Sunday| 158|
 70.45833333| 1012.666667| 295.56| 1.875| Saturday| 172|
 64.95833333| 1014.041667| 289.4675| 0.916666667| Saturday| 139|
 63.625| 1024.25|291.6047083| 0.416666667| Thursday| 137|
 79.625| 1013.125| 287.545| 1.0| Wednesday| 128|
 53.16666667| 1016.708333| 284.832329| 1.666666667| Friday| 140|
 77.08333333| 1014.958333|291.0075729| 0.791666667| Saturday| 84|
 58.29166667| 1027.333333|284.1869375| 0.75| Friday| 143|
 29.83333333| 1027.826087|280.8922917| 0.708333333| Wednesday| 97|
 68.75| 1010.625|285.9004167| 1.0| Friday| 148|
 55.83333333| 1014.041667| 285.62125| 1.0| Saturday| 74|
 68.08333333| 1013.25|293.2074063| 0.416666667| Sunday| 121|
 63.25| 1011.791667|297.0499436| 1.666666667| Thursday| 131|
 69.70833333| 1024.75|284.1489167| 0.791666667| Saturday| 143|
 80.20833333| 1010.208333| 295.91| 2.333333333| Tuesday| 150|
 70.41666667| 1013.25|291.3427083| 0.791666667| Wednesday| 110|
 78.33333333| 1020.333333|286.4395833| 0.166666667| Tuesday| 103|
+------------+------------+-----------+--------------+-----------+---------+
only showing top 20 rows

In [0]:
# Checking missing values in the dataset
spark.sql("select * from collision WHERE AVE_HUMUDITY IS NULL \
OR AVE_PRESSURE IS NULL \
OR AVE_TEMP IS NULL \
OR AVE_WIND_SPEED IS NULL \
OR DAY_OF_WEEK IS NULL").show()

+----+------------+------------+--------+--------------+-----------+---------+
DATE|AVE_HUMUDITY|AVE_PRESSURE|AVE_TEMP|AVE_WIND_SPEED|DAY_OF_WEEK|COLLISION|
+----+------------+------------+--------+--------------+-----------+---------+
+----+------------+------------+--------+--------------+-----------+---------+

In [0]:
data.count()

Out[35]: 1282

In [0]:
# Create a 70-30 train test split

train_data,test_data=data.randomSplit([0.7,0.3], seed=2020)

#### Data Pipeline for Model Building

In [0]:
# Import the required libraries for Pipeline

from pyspark.ml.feature import VectorAssembler,StringIndexer,StandardScaler
from pyspark.ml import Pipeline

In [0]:
# Use StringIndexer to convert the categorical columns to hold numerical data

day_of_week_indexer = StringIndexer(inputCol='DAY_OF_WEEK',outputCol='day_of_week_index',handleInvalid='keep')

In [0]:
# Vector assembler is used to create a vector of input features

assembler = VectorAssembler(inputCols=['AVE_HUMUDITY','AVE_PRESSURE','AVE_TEMP',
                                       'AVE_WIND_SPEED','day_of_week_index'],
                            outputCol="unscaled_features")

In [0]:
# Standard scaler is used to scale the data for the linear regression to perform well on the training data

scaler = StandardScaler(inputCol="unscaled_features",outputCol="features")

In [0]:
# Pipeline is used to pass the data through indexer and assembler simultaneously. Also, it helps to pre-rocess the test data
# in the same way as that of the train data

pipe = Pipeline(stages=[day_of_week_indexer,assembler,scaler])

In [0]:
# Fit Train Data
fitted_pipe=pipe.fit(train_data)

In [0]:
# Transform Train Data
train_data=fitted_pipe.transform(train_data)

In [0]:
train_data.show()

+------------+------------+-----------+--------------+-----------+---------+-----------------+--------------------+--------------------+
AVE_HUMUDITY|AVE_PRESSURE| AVE_TEMP|AVE_WIND_SPEED|DAY_OF_WEEK|COLLISION|day_of_week_index| unscaled_features| features|
+------------+------------+-----------+--------------+-----------+---------+-----------------+--------------------+--------------------+
 14.58333333| 1012.666667|296.0891667| 3.333333333| Saturday| 130| 5.0|[14.58333333,1012...|[0.85878785642582...|
 14.70833333| 1003.166667| 299.34375| 2.416666667| Monday| 76| 0.0|[14.70833333,1003...|[0.86614889519687...|
 15.54166667| 1022.333333|287.6029167| 1.208333333| Tuesday| 110| 6.0|[15.54166667,1022...|[0.91522248739643...|
 16.58333333| 1004.916667|300.2038333| 0.875| Thursday| 128| 3.0|[16.58333333,1004...|[0.97656447676257...|
 16.79166667| 1022.5|293.1774887| 0.833333333| Wednesday| 171| 4.0|[16.79166667,1022...|[0.98883287510690...|
 18.2173913| 1020.166667|291.4064583| 0.625| Sunday| 125| 1.0|[18.2173913,1020....|[1.07279138933303...|
 18.375| 1006.625| 297.522625| 1.25| Tuesday| 95| 6.0|[18.375,1006.625,...|[1.08207269934387...|
 18.52173913| 1012.291667|295.6941667| 0.083333333| Sunday| 101| 1.0|[18.52173913,1012...|[1.09071391874514...|
 19.83333333| 1019.458333| 291.86375| 0.541666667| Thursday| 106| 3.0|[19.83333333,1019...|[1.16795148480978...|
 19.91304348| 1016.782609| 290.34625| 0.416666667| Thursday| 133| 3.0|[19.91304348,1016...|[1.17264548084655...|
 20.20833333| 1012.666667|290.2345833| 0.791666667| Friday| 108| 2.0|[20.20833333,1012...|[1.19003460112292...|
 20.79166667| 1013.0|289.2279167| 1.708333333| Tuesday| 87| 6.0|[20.79166667,1013...|[1.22438611578040...|
 21.5| 1014.208333|288.1972917| 0.916666667| Monday| 101| 0.0|[21.5,1014.208333...|[1.26609866862003...|
 21.68181818| 1019.5| 288.47875| 0.666666667| Monday| 68| 0.0|[21.68181818,1019...|[1.27680563399812...|
 22.375| 1012.958333|301.8384697| 0.916666667| Monday| 173| 0.0|[22.375,1012.9583...|[1.31762594001736...|
 22.875| 1014.25|294.4370522| 2.833333333| Saturday| 204| 5.0|[22.875,1014.25,2...|[1.34707009510155...|
 22.95833333| 1034.708333|282.2548958| 0.791666667| Monday| 102| 0.0|[22.95833333,1034...|[1.35197745408595...|
 23.375| 1014.75|294.7284896| 0.958333333| Monday| 110| 0.0|[23.375,1014.75,2...|[1.37651425018573...|
 23.45454545| 1009.916667|300.4961667| 1.041666667| Friday| 136| 2.0|[23.45454545,1009...|[1.38119854731782...|
 23.52173913| 1017.166667|289.6554167| 0.291666667| Thursday| 105| 3.0|[23.52173913,1017...|[1.38515546958701...|
+------------+------------+-----------+--------------+-----------+---------+-----------------+--------------------+--------------------+
only showing top 20 rows

In [0]:
# Transform the test data using the model to predict the duration

test_data=fitted_pipe.transform(test_data)

#### ---------------------------------------------------------------------------------------------------------------------------------------------

#### Model#1: Linear Regression Model

###### Model#1: Building Linear Regression Model

In [0]:
# Import the required libraries for LR

from pyspark.ml.regression import LinearRegression

In [0]:
# Create an object for the Linear Regression model

lr_model = LinearRegression(labelCol='COLLISION')

In [0]:
# Fit the model on the train data

fit_model = lr_model.fit(train_data.select(['features','COLLISION']))

In [0]:
# Store the results in a dataframe

results = fit_model.transform(test_data)

In [0]:
# Result Display
results.select(['COLLISION','prediction']).show()

+---------+------------------+
COLLISION| prediction|
+---------+------------------+
 176|160.29325193858313|
 118| 129.772782788184|
 134|137.44955520140948|
 160|132.46745454623704|
 202|147.14164771070276|
 176|139.56126757103635|
 113|146.55269681793675|
 173|146.37195153225153|
 104|136.98017402548444|
 128| 144.7220010007142|
 145|143.13942056211724|
 216|144.98542128659827|
 82|138.97829463997022|
 166|144.49755285630965|
 119|136.58043557731227|
 155|132.18638443501314|
 155| 133.3319110881212|
 152| 135.2507322232261|
 147| 128.4564201191436|
 111| 131.9637249151741|
+---------+------------------+
only showing top 20 rows

###### Model#1: Evaluating the Linear Regression Model

In [0]:
test_results = fit_model.evaluate(test_data)

In [0]:
# Display Residual
test_results.residuals.show()

+-------------------+
 residuals|
+-------------------+
 15.706748061416874|
-11.772782788184003|
-3.4495552014094812|
 27.532545453762964|
 54.858352289297244|
 36.43873242896365|
 -33.55269681793675|
 26.62804846774847|
-32.980174025484445|
-16.722001000714187|
 1.8605794378827625|
 71.01457871340173|
-56.978294639970215|
 21.502447143690347|
-17.580435577312272|
 22.81361556498686|
 21.66808891187881|
 16.749267776773905|
 18.543579880856413|
 -20.96372491517411|
+-------------------+
only showing top 20 rows

In [0]:
# Display RMSE
rmse_lr = test_results.rootMeanSquaredError
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse_lr)

Root Mean Squared Error (RMSE) on test data = 30.404

The root mean squared error is very high indicating that the models prediction is on the poorer side

In [0]:
# Display R2
test_results.r2

Out[54]: 0.0705539409802437

The r-squared value implies that the model explains only about 7% variance

#### Model#2: RandomForest Regression Model

###### Model#2: Building RandomForest Regression Model

In [0]:
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator

In [0]:
rf = RandomForestRegressor(labelCol="COLLISION",featuresCol="features")

In [0]:
rf_model = rf.fit(train_data)

In [0]:
predictions = rf_model.transform(test_data)

In [0]:
predictions.select("COLLISION","prediction").show()

+---------+------------------+
COLLISION| prediction|
+---------+------------------+
 176|146.52927687072503|
 118|114.00624598390493|
 134| 133.726532031697|
 160|123.20554038907919|
 202| 144.7466109649732|
 176| 131.2453826655389|
 113|142.82350807406607|
 173|138.81854256854257|
 104|129.74252571258427|
 128| 139.1280372029033|
 145|140.36962750935197|
 216| 153.4373942622661|
 82| 134.7323958305617|
 166|136.25654987932296|
 119| 134.9034494506101|
 155|130.08413150397064|
 155|132.62057120000662|
 152|137.85345094178018|
 147|149.51990754836456|
 111|131.25250926146242|
+---------+------------------+
only showing top 20 rows

###### Model#2: Evaluating RandomForest Regression Model

In [0]:
evaluator = RegressionEvaluator(labelCol="COLLISION", predictionCol="prediction", metricName="rmse")

In [0]:
rmse_rf = evaluator.evaluate(predictions)

In [0]:
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse_rf)

Root Mean Squared Error (RMSE) on test data = 28.7169

#### Model#3: Decision Tree Regression Model

###### Model#3: Building DecisionTree Regression Model

In [0]:
from pyspark.ml.regression import DecisionTreeRegressor

In [0]:
dt = DecisionTreeRegressor(labelCol="COLLISION",featuresCol="features")

In [0]:
train_data.show()

+------------+------------+-----------+--------------+-----------+---------+-----------------+--------------------+--------------------+
AVE_HUMUDITY|AVE_PRESSURE| AVE_TEMP|AVE_WIND_SPEED|DAY_OF_WEEK|COLLISION|day_of_week_index| unscaled_features| features|
+------------+------------+-----------+--------------+-----------+---------+-----------------+--------------------+--------------------+
 14.58333333| 1012.666667|296.0891667| 3.333333333| Saturday| 130| 5.0|[14.58333333,1012...|[0.85878785642582...|
 14.70833333| 1003.166667| 299.34375| 2.416666667| Monday| 76| 0.0|[14.70833333,1003...|[0.86614889519687...|
 15.54166667| 1022.333333|287.6029167| 1.208333333| Tuesday| 110| 6.0|[15.54166667,1022...|[0.91522248739643...|
 16.58333333| 1004.916667|300.2038333| 0.875| Thursday| 128| 3.0|[16.58333333,1004...|[0.97656447676257...|
 16.79166667| 1022.5|293.1774887| 0.833333333| Wednesday| 171| 4.0|[16.79166667,1022...|[0.98883287510690...|
 18.2173913| 1020.166667|291.4064583| 0.625| Sunday| 125| 1.0|[18.2173913,1020....|[1.07279138933303...|
 18.375| 1006.625| 297.522625| 1.25| Tuesday| 95| 6.0|[18.375,1006.625,...|[1.08207269934387...|
 18.52173913| 1012.291667|295.6941667| 0.083333333| Sunday| 101| 1.0|[18.52173913,1012...|[1.09071391874514...|
 19.83333333| 1019.458333| 291.86375| 0.541666667| Thursday| 106| 3.0|[19.83333333,1019...|[1.16795148480978...|
 19.91304348| 1016.782609| 290.34625| 0.416666667| Thursday| 133| 3.0|[19.91304348,1016...|[1.17264548084655...|
 20.20833333| 1012.666667|290.2345833| 0.791666667| Friday| 108| 2.0|[20.20833333,1012...|[1.19003460112292...|
 20.79166667| 1013.0|289.2279167| 1.708333333| Tuesday| 87| 6.0|[20.79166667,1013...|[1.22438611578040...|
 21.5| 1014.208333|288.1972917| 0.916666667| Monday| 101| 0.0|[21.5,1014.208333...|[1.26609866862003...|
 21.68181818| 1019.5| 288.47875| 0.666666667| Monday| 68| 0.0|[21.68181818,1019...|[1.27680563399812...|
 22.375| 1012.958333|301.8384697| 0.916666667| Monday| 173| 0.0|[22.375,1012.9583...|[1.31762594001736...|
 22.875| 1014.25|294.4370522| 2.833333333| Saturday| 204| 5.0|[22.875,1014.25,2...|[1.34707009510155...|
 22.95833333| 1034.708333|282.2548958| 0.791666667| Monday| 102| 0.0|[22.95833333,1034...|[1.35197745408595...|
 23.375| 1014.75|294.7284896| 0.958333333| Monday| 110| 0.0|[23.375,1014.75,2...|[1.37651425018573...|
 23.45454545| 1009.916667|300.4961667| 1.041666667| Friday| 136| 2.0|[23.45454545,1009...|[1.38119854731782...|
 23.52173913| 1017.166667|289.6554167| 0.291666667| Thursday| 105| 3.0|[23.52173913,1017...|[1.38515546958701...|
+------------+------------+-----------+--------------+-----------+---------+-----------------+--------------------+--------------------+
only showing top 20 rows

In [0]:
dt_model = dt.fit(train_data)

In [0]:
dt_predictions = dt_model.transform(test_data)

In [0]:
dt_predictions.select("COLLISION", "prediction").show()

+---------+------------------+
COLLISION| prediction|
+---------+------------------+
 176|154.42105263157896|
 118| 111.0909090909091|
 134| 140.4277108433735|
 160| 127.5925925925926|
 202| 135.0625|
 176| 135.0625|
 113| 130.2|
 173| 76.0|
 104|153.28571428571428|
 128|153.28571428571428|
 145| 140.4277108433735|
 216|194.33333333333334|
 82| 140.4277108433735|
 166| 140.4277108433735|
 119| 140.4277108433735|
 155| 127.5925925925926|
 155| 127.5925925925926|
 152| 140.4277108433735|
 147|153.28571428571428|
 111| 127.5925925925926|
+---------+------------------+
only showing top 20 rows

###### Model#3: Evaluating DecisionTree Regression Model

In [0]:
evaluator = RegressionEvaluator(labelCol="COLLISION", predictionCol="prediction", metricName="rmse")
rmse_dt = evaluator.evaluate(dt_predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse_dt)

Root Mean Squared Error (RMSE) on test data = 29.2589

#### Model#4: Gradient-Boosted Tree Regression Model

###### Model#4: Building Gradient-Boosted Tree Regression Model

In [0]:
from pyspark.ml.regression import GBTRegressor

In [0]:
gbt = GBTRegressor(labelCol="COLLISION",featuresCol="features", maxIter=10)

In [0]:
gtb_model = gbt.fit(train_data)

In [0]:
gtb_predictions = gtb_model.transform(test_data)

In [0]:
gtb_predictions.select("COLLISION", "prediction").show()

+---------+------------------+
COLLISION| prediction|
+---------+------------------+
 176|151.74604202651818|
 118| 98.32733787478088|
 134| 125.4712401091636|
 160|114.82902137646438|
 202|142.92629690294862|
 176|128.03832121260834|
 113|162.50103310499634|
 173| 90.79352511806417|
 104| 139.9863341529199|
 128| 145.2988305205624|
 145| 147.1728981761319|
 216| 192.8866148285349|
 82| 131.3490058705832|
 166| 150.0826256412391|
 119|140.06328480885324|
 155|125.73082860087416|
 155|127.22816655807233|
 152|163.30137566894913|
 147|157.70076062158734|
 111|125.73082860087416|
+---------+------------------+
only showing top 20 rows

###### Model#4: Evaluating Gradient-Boosted Tree Regression Model

In [0]:
evaluator = RegressionEvaluator(labelCol="COLLISION", predictionCol="prediction", metricName="rmse")
rmse_gtb = evaluator.evaluate(gtb_predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse_gtb)

Root Mean Squared Error (RMSE) on test data = 29.0547

#### Model Summary:

In [0]:
print("Model#1: Linear Regression Model: Root Mean Squared Error (RMSE) on test data = %g" % rmse_lr)
print("Model#2: RandomForest Regression Model: Root Mean Squared Error (RMSE) on test data = %g" % rmse_rf)
print("Model#3: DecisionTree Regression Model: Root Mean Squared Error (RMSE) on test data = %g" % rmse_dt)
print("Model#4: Gradient-Boosted Tree Regression Model: Root Mean Squared Error (RMSE) on test data = %g" % rmse_gtb)

Model#1: Linear Regression Model: Root Mean Squared Error (RMSE) on test data = 30.404
Model#2: RandomForest Regression Model: Root Mean Squared Error (RMSE) on test data = 28.7169
Model#3: DecisionTree Regression Model: Root Mean Squared Error (RMSE) on test data = 29.2589
Model#4: Gradient-Boosted Tree Regression Model: Root Mean Squared Error (RMSE) on test data = 29.0547